In [9]:
import cv2
from ultralytics import YOLO
from datetime import datetime
import mysql.connector

# ------------------ DATABASE ------------------
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="crowd_monitor"
)
cursor = db.cursor()

# ------------------ LOAD BETTER MODEL ------------------
model = YOLO("yolov8s.pt")   # more accurate than yolov8n

# ------------------ CAMERA ------------------
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

max_people_seen = 0
unique_ids = set()   # Track unique people

print("\nHigh Accuracy Crowd Monitoring Started...\nPress Q to Stop\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Use tracking instead of simple detection
    results = model.track(
        frame,
        persist=True,
        conf=0.6,
        imgsz=640,
        classes=[0]   # detect only person
    )

    people_count = 0

    if results[0].boxes.id is not None:

        boxes = results[0].boxes
        ids = boxes.id.cpu().numpy().astype(int)

        for box, track_id in zip(boxes.xyxy, ids):

            x1, y1, x2, y2 = map(int, box)

            people_count += 1
            unique_ids.add(track_id)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, f"ID:{track_id}",
                        (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0,255,0), 1)

    # Update session max
    if people_count > max_people_seen:
        max_people_seen = people_count

    total_unique_people = len(unique_ids)

    # Display
    cv2.putText(frame, f"Current Count: {people_count}",
                (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                0.8, (0, 0, 255), 2)

    cv2.putText(frame, f"Session Max: {max_people_seen}",
                (20, 80), cv2.FONT_HERSHEY_SIMPLEX,
                0.8, (255, 0, 0), 2)

    cv2.putText(frame, f"Total Unique (Session): {total_unique_people}",
                (20, 120), cv2.FONT_HERSHEY_SIMPLEX,
                0.8, (0, 150, 255), 2)

    # Terminal
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{current_time}] Current: {people_count} | Max: {max_people_seen} | Unique: {total_unique_people}")

    # Save to database
    sql = "INSERT INTO people_log (time, current_count, total_detected) VALUES (%s, %s, %s)"
    values = (current_time, people_count, total_unique_people)
    cursor.execute(sql, values)
    db.commit()

    cv2.imshow("High Accuracy Crowd Monitoring", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print("\nFinal Unique People (Session):", total_unique_people)

cap.release()
cv2.destroyAllWindows()
cursor.close()
db.close()


High Accuracy Crowd Monitoring Started...
Press Q to Stop

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.5/1.5 MB 1.9 MB/s  0:00:00:01

requirements: AutoUpdate success  5.5s
WARNING requirements: Restart runtime or rerun command for updates to take effect


0: 480x640 1 person, 335.0ms
Speed: 5.4ms preprocess, 335.0ms inference, 12.7ms postprocess per image at shape (1, 3, 480, 640)
[2026-02-25 21:54:31] Current: 1 | Max: 1 | Unique: 1

0: 480x640 1 person, 203.7ms
Speed: 2.8ms preprocess, 203.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
[2026-02-25 21:54:31] Current: 1 | Max: 1 | Unique: 1

0: 480x640 1 person, 172.4ms
Speed: 1.6ms preprocess, 172.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
[2026-02-25 21:54:31] Current: 1 | Max: 1 | Unique: 1

0: 480x640 1 person, 192.2ms
Speed: 2.0ms preprocess, 192.2ms inference, 1.3ms postprocess per image at

In [8]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.
